Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Hegyi Gáspár András"
COLLABORATORS = ""

---

# Assignment 1: Fisher's and $\chi^2$-test

In [2]:
# setup
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import math

def assert_approx_equal(value1,value2):
    assert np.isclose(value1,value2)

A web authoring company uses A/B testing to select the best website design. A/B testing involves creating multiple versions (*A* and *B*) of the website in a campaign to test their effectiveness. Marketers split their audience and direct them to different versions to determine which performs better.


**Workflow of A/B testing:**
1. **Hypothesis:** Formulate a hypothesis about changes that might improve the click-through rate.
2. **Variations:** Create two versions of the original site with specific changes.
3. **Traffic Split:** Divide incoming traffic equally between the two website versions.
4. **Testing Duration:** Run the test until statistically significant results are obtained.
5. **Data Analysis:** Analyze the data to determine which version performs better.

When comparing two phenomena (like the efficacy of drugs), we use statistical tests. For comparing, e.g., means of two continuous variables, Student's t-test is used. When we observe counts of occurrences of phenomena, we usually use the $\chi^2$-test. 
However, Fisher's exact test is more suitable when the observed counts are small and in the form of a four-field contingency table. 

A/B testing allows the company to take a scientific approach to marketing.
The company prepared two versions of the website, denoted as *A* and *B*. Users who visit the web page are shown one of the two new versions of the page. We will compare the efficacy of the versions *A* and *B*. We observed the number of clicks within the site.
The following contingency table summarizes the results. In the table, $N$ denote the total number of visitors, $a$ is the number of visitors who saw the version *A* and followed a link
within *A*, $b$ is the number of visitors of version *A* that did not follow 
any link within the website, ...


#### Table 1
|          | Click YES | Click NO | | Row sum |	
|----------|-----------|----------|-|---------|
| Sample A |   $a$     |   $b$    | | $a+b$   |
| Sample B |   $c$     |   $d$    | | $c+d$   |
|__________|___________|__________| |_________|
| Sum      |  $a+c$    |  $b+d$   | | $N$     |

At first, we must formulate the so-called null hypothesis.

> Samples $A$ and $B$ were taken from the same probability distribution, and the differences between them
> were caused by accidents only. In other words, the efficacies of both versions $A$ and $B$ are the same. 

## Task 1 
Assuming that the null hypothesis is true, **derive the formula for computing the probability** that the table with results will have the same values as in *Table 1* for given values $a$, $b$, $c$, and $d$ ($N=a+b+c+d$). 

**Remarks:**

* Stating that the probability corresponds to the hypergeometric probability distributions is insufficient. You should explain the meaning of all binomial coefficients or factorials in the formulas you will use!
* You can write the complete derivation of the formula for the probability of *Table 1* by hand and submit it as a scanned picture (or an image captured by, e.g., a mobile phone) in a separate file.
* You can get at most **3 points** for this part.

**Here, you can write your answer or the file name where the derivation is.**

1. Assume the null hypothesis, that samples A and B are taken from the same probability distribution. We can reformulate the question to what is the probability that `a` people clicked yes on website `A` with the given marginal values. Since the marginal values are known, the other table values (`b,c,d`) are determined.

2. Suppose the samples are labeled. If the samples are taken uniformly, it means that there are $n!$ possible orderings if the samples. 

3. Count the possibilities that there are exactly `a` people clicking on website `A`. Let's say we first get all the samples that click yes, which is (a+c). This means we have to count the possible ways to have `a` number of samples belonging to website `A`. This can be obtained by the combination $(a+c)\choose a$.

4. By the same idea, the possible ways of having `b` people not clicking on website `A` from $b+d$ people who did not click is $(b+d)\choose b$.

5. `a` and `b` samples are chosen for website `A`, however since the samples are labeled there are $(a+b)!$ ways of ordering them. Similarly for website `B`, there are $(c+d)!$ ways of ordering them.

6. The final probability is the product of the combinations and permutations, over the permutations of all of the elements. This is the hypergeometric distribution with the variable `a`.  
$$P(a_{11} = a)=\frac{{(a+c)\choose a} {(b+d)\choose b} (a+b)!(c+d)!}{n!} = \frac{{(a+c)\choose a}{(b+d)\choose b}}{n\choose (a+b)}$$  

7. Replacing the marginal values with $s_1,s_2$ for the columns and $r_1,r_2$ for the rows,we get the following equation:
$$P(a_{11} = a)= \frac{{s_1\choose a}{s_2\choose b}}{n\choose r_1}=\frac{{s_1\choose a}{s_2\choose b}r_1!r_2!}{n!}= \frac{s_1!s_2!r_1!r_2!}{n!a!\left(s_1-a\right)!b!\left(s_2-b\right)!} = \frac{s_1!s_2!r_1!r_2!}{n!a!b!c!d!} $$

## Task 2
In a campaign, the following counts were observed:

#### *Table 2*
|          | Click YES | Click NO | | Row sum |	
|----------|-----------|----------|-|---------|
| Sample A |   4       |   10     | | 14      |
| Sample B |   7       |   3      | | 10      |
|__________|___________|__________| |_________|
| Sum      |  11       |   13     | | 24      |

Implement function `table_probability(t)` that computes the probability of *Table 1* (`t` is a 2-by-2 numpy array with the values $a$, $b$, $c$ and $d$ in two rows and two columns) when assuming the null hypothesis is valid, using the formula you have derived in **Task 1**. 

In [3]:
def getMarginals(t):
    """
    Returns
    -------
    s1,s2,r1,r2
    """
    row_sums = np.sum(t,axis=1)
    col_sums = np.sum(t,axis = 0)
    return col_sums[0],col_sums[1],row_sums[0],row_sums[1]

#The equation from task 1, rewritten with the notation(k,N,K,n) of the hypergeometric distribution
def hypergeometric(k,N,K,n):
    return math.comb(K,k)*math.comb(N-K,n-k)/math.comb(N,n)

def table_probability(t):
    a = t[0,0]
    s1,s2,r1,r2 = getMarginals(t)
    N = np.sum(t)
    k = a
    K = s1
    n = r1
    return hypergeometric(k,N,K,n)

Using the function, calculate the probability of *Table 2*. Additionally, your implementation should pass all the tests below and some additional hidden tests (**1 point for this part**).

In [4]:
table = np.array([[5,5], [5,5]])
assert_approx_equal(table_probability(table),0.343718)

table_2 = np.array([[4, 10], [7, 3]])
print(f"{table_2} has probability {table_probability(table_2)}")


[[ 4 10]
 [ 7  3]] has probability 0.04812222371786243


## Task 3

The difference between versions *A* and *B* of the website is evident. Is this difference statistically significant? That is, assuming that both samples *A* and *B* are from the same probability distribution, what is the probability that two samples differ to the same or even higher extent? If this probability is small, e.g., at most $\alpha=0.05$, we can state with high confidence $(1−\alpha)=0.95$ that the null hypothesis is not valid. Based on the marginal sums ($a+b$, $c+d$, $a+c$, and $b+d$), we can easily compute that the expected value of the field $a$ is approximately $6.16$. The notion of "differing to the same or even greater extent" can be understood in two ways:

1. one-sided &ndash; only the values of $a$ that are on one side from the expected value; in our case, the values 8 and 9, or
2. two-sided &ndash; all the values of $a$ such that $|a−6.16|\ge 8−6.16$; in our case, the values 0, 1, 2, 3, 4, 8, and 9.
  
In case 1, we use a one-tailed test; in case 2, we use a two-tailed test.

Answer the following question (**1 point**)

> **Question:** In general (with sufficient data), which of the four combinations of tests {one-tiled, two-tailed}×
{Fisher's test, χ2-test} are meaningful?

**Your answer goes here.**

Two-tailed tests should be used when the distribution we are working with is symmetric, and is converging to 0 on both ends (having 2 tails), and if the distribution is assymetric it is better to use a one-tailed test. A one-tailed test can also be used on one end of a symmetric distribution, however for that we have to know the direction of change in the data, and we have to make sure that checking only one f the tails makes sense.  
  
So from the above combinations, for the Fisher's test is makes sense to use both one-tailed and two-tailed tests, since it uses a hypergeometric distribution, and for the $\chi^2$ a one-tailed should be used, since the $\chi^2$ distribution is assymetric. 

Using function `table_probability`, implement the function `Fisher_p_value(table, alternative)` that for the contingency table `table` of dimension 2 $\times$ 2 computes the p-value of Fisher's test. The parameter `alternative` can have only two values:
1. For `alternative` equal to 'two-tailed', the function returns the p-value of the two-tailed Fisher's test.
2. For `alternative` equal to 'one-tailed', the function returns the p-value of the one-tailed Fisher's test, i.e., the probability that 
   the observed counts are as seen in `table` or are more extreme &ndash; further from the expected ones *in the same 
   direction* from the expected counts as `table`. If `table` contains exactly the expected counts, we will consider the direction for the values in the upper left corner less or equal to $a$.

In [11]:
def tableProbabilities(t):
    """
    Parameter
    -----------
    A 2x2 contingency table

    Returns
    ------------
    A list of the probabilities of all the possible tables with the given marginal values.
    """
    marginals= getMarginals(t)
    minMarginal = np.argmin(marginals)
    s1,s2,r1,r2 = marginals
    #Flip rows, columns and the corresponding marginals
    if(minMarginal == 1):
        t = t[:,[1,0]]
        s1,s2 = s2,s1
    elif(minMarginal == 3):
        t = t[[1,0]]
        r1,r2 = r2,r1
    N = np.sum(t)
    return np.array([table_probability(np.array([[k,r1-k],[s1-k,N+k-s1-r1]])) for k in range(0,marginals[minMarginal]+1)])

def Fisher_p_value(table, alternative='two-tailed'):
    """
    Parameters
    -----------
    table : A 2x2 cotingency table

    alternative: The type of p-value to be calculated, values are "one-tailed" or "two-tailed"

    Returns
    ---------
    The p-value of the given contingency table
    """
    if table.any()<0:
        ValueError("Table can not contain negative value!")
    allProbabilities = tableProbabilities(table)
    tableProbability = table_probability(table)
    if alternative == 'two-tailed':
        return sum(allProbabilities[np.where(allProbabilities<=tableProbability)])
    ## Else get the minimum value of the table, and sum up all of the tables, where that 
    ## particular cell contains a lower or equal value to the original value (same as in the slides)
    minVal = np.min(table)
    minPos = np.argwhere(table == minVal)
    # Flip rows and columns, so minimum value is in a_11
    if minPos[0,0] == 1:
        table = table[[1,0]]
    if minPos[0,1] == 1:
        table = table[:,[1,0]]
    a11,a12,a21,a22 = table[0,0],table[0,1], table[1,0], table[1,1]
    tailProbs = [table_probability(np.array([[a11-k,a12+k],[a21+k,a22-k]])) for k in range(0,a11+1)]
    return sum(tailProbs)


Now, we can compute p-values for both alternatives of the Fisher's test for *Table 2*. In the following cell, the function `Fisher_p_value` will be tested (including some hidden tests) and applied on `table_2` (**2 points** of the score).

In [12]:
t = np.array([[5,5], [5,5]])
alternative = 'two-tailed'
p = Fisher_p_value(t, alternative)
print(f"p-value of {alternative} Fisher's test for the table\n{t=}\n is {p}")
assert_approx_equal(p, 1.0)

print('_'*30)

t = np.array([[5,5], [5,5]])
alternative = 'one-tailed'
p = Fisher_p_value(t, alternative)
print(f"p-value of {alternative} Fisher's test for the table\n{t=}\n is {p}")
assert_approx_equal(p, 0.6718591)

print('='*20)

t = np.array([[38,5], [20,9]])
alternative = 'one-tailed'
p = Fisher_p_value(t, alternative)
print(f"p-value of {alternative} Fisher's test for the table\n{t=}\n is {p}")
assert_approx_equal(p, 0.042128934)

alternative = 'two-tailed'
p = Fisher_p_value(t, alternative)
print(f"p-value of {alternative} Fisher's test for the table\n{t=}\n is {p}")
assert_approx_equal(p, 0.0667809135)

print("="*60 + "\n" + "="*60)
one_tailed_Fisher_p_value = Fisher_p_value(table_2, 'one-tailed')
two_tailed_Fisher_p_value = Fisher_p_value(table_2, 'two-tailed')
print(f"For the table\n{table_2}")
print(f"The p-value of the Fisher's one-tailed test is {one_tailed_Fisher_p_value}")
print(f"The p-value of the Fisher's two-tailed test is {two_tailed_Fisher_p_value}")


p-value of two-tailed Fisher's test for the table
t=array([[5, 5],
       [5, 5]])
 is 1.0
______________________________
p-value of one-tailed Fisher's test for the table
t=array([[5, 5],
       [5, 5]])
 is 0.6718591006516703
p-value of one-tailed Fisher's test for the table
t=array([[38,  5],
       [20,  9]])
 is 0.04212893437210189
p-value of two-tailed Fisher's test for the table
t=array([[38,  5],
       [20,  9]])
 is 0.06678091352515701
For the table
[[ 4 10]
 [ 7  3]]
The p-value of the Fisher's one-tailed test is 0.05505451608561045
The p-value of the Fisher's two-tailed test is 0.09530219410418629


While ignoring the requirement that the χ2-test can be used only if all expected counts in the contingency table are at least 5, for all meaningful combinations of the χ2-test for *Table 2* compute χ2-statistics and its corresponding p-value.

For computing the χ2-test use suitable functions from the Python `scipy` module like `scipy.stats.chi2.pdf()`, `scipy.stats.chi2.cdf()`, `scipy.stats.chi2.sf()`, and `scipy.stats.chi2.isf()`. Your code should end with storing the value of the χ2-statistics of any of the meaningful combinations into variable `x2_stat` and its corresponsing p-value in the variable `x2_p_value`. 

In [13]:
# here goes your code for computing  χ2-test using suitable functions 
# from the Python scipy module like scipy.stats.chi2.cdf(), 
# scipy.stats.chi2.sf() and scipy.stats.chi2.isf()
# BUT NOT scipy.stats.chi2_contingency()
def expectedTable(table):
    s1,s2,r1,r2 = getMarginals(table)
    n = np.sum(table)
    return np.outer([r1,r2],[s1,s2])/n

def chi2Stat(table):
    expected = expectedTable(table)
    return np.sum(np.power(table-expected,2.0)/expected)    

x2_stat = chi2Stat(table_2)

dof = (table_2.shape[0]-1)*(table_2.shape[1]-1)
x2_p_value = 1-scipy.stats.chi2.cdf(x2_stat,dof) 


 In the following cell, your results (`x2_stat` and `x2_p_value`) will be evaluated (**1 point**).

In [14]:
# here, your solution will be evaluated
print(f"{x2_stat=}")
print(f"{x2_p_value=}")

x2_stat=4.032767232767235
x2_p_value=0.04462468800071262


Of course, `scipy` contains functions for computing Fisher's exact test and χ2-test. Compute the above tests for *Table 2* using the functions `scipy.stats.fisher_exact()` and `scipy.stats.chi2_contingency()`  (**1 point**).

In [15]:
#Fisher's tests
fishersTwoSided = scipy.stats.fisher_exact(table_2)
fishersLess = scipy.stats.fisher_exact(table_2,"less")
fishersGreater = scipy.stats.fisher_exact(table_2,"greater")
print(f"The two-tailed Fisher's test on table2 with statistics {fishersTwoSided.statistic} and p-value {fishersTwoSided.pvalue}")
print(f"The less one-tailed Fisher's test on table2 with statistics {fishersLess.statistic} and p-value {fishersLess.pvalue}")
print(f"The greater one-sided Fisher's test on table2 with statistics: {fishersGreater.statistic} and p-value {fishersGreater.pvalue}")

The two-tailed Fisher's test on table2 with statistics 0.17142857142857143 and p-value 0.0953021941041863
The less one-tailed Fisher's test on table2 with statistics 0.17142857142857143 and p-value 0.05505451608561045
The greater one-sided Fisher's test on table2 with statistics: 0.17142857142857143 and p-value 0.9930677076322519


In [16]:
#X2 tests
x2 = scipy.stats.chi2_contingency(table_2,correction = False)
print(f"The X2 test's statistics without Yates correction on table2 is {x2.statistic} and p-value is {x2.pvalue}")

x2Corrected = scipy.stats.chi2_contingency(table_2)
print(f"The corrected X2 test's statistics on table2 is {x2Corrected.statistic} and p-value is {x2Corrected.pvalue}")


The X2 test's statistics without Yates correction on table2 is 4.032767232767235 and p-value is 0.04462468800071265
The corrected X2 test's statistics on table2 is 2.5366633366633375 and p-value is 0.11122961808080513


## Final evaluation

1. State explicitly whether we can or cannot reject the null hypothesis for each test you have performed for *Table 2*.
2. Further, compare the results obtained with your implementation of the tests and the results obtained when using the functions 
   `scipy.stats.fisher_exact()` and `scipy.stats.chi2_contingency()`. If there are any differences, explain them (**1 point**). 

**Answer**

1. Since `Table 2` is a 2X2 contingency table, I am using the results from the Fisher's test to determine whether we can or cannot reject the null hypothesis. In case of Fisher's test, if the p-value is lower than the significance level $\alpha$, the null hypothesis is rejected. With the two-tailed p-value $\approx 0.095$, and significance level $\alpha = 0.05$, we can not reject the null hypothesis, which means that the tests on the clicks on the different websites were drawn from the same distribution, i.e. the efficacies of the two websites are the same. We can arrive to the same conclusion with the one-tailed test.
  
2. The results that are obtained from the scipy.stats package have multiple different arguments. 
    - The `scipy.stats.chi2_contingency()` has an argument for the Yates correction which moves the table's values by 0.5 in order to correct the error introduced by assuming that the table probabilities can be described using a continuous $\chi^2$ distribution. Setting the argument to false, the results are the same as in my implementation.
    - The `scipy.stats.fisher_exact()` has argument $alternative$ for the calculation of the p-value. These are two-tailed, less, or greater. The latter two options decide the one-tailed p-value's direction. Choosing the value two-tailed , or the less has the same results as my implementation with the corresponding alternatives.
